# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from apitoken import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. (I changed it to two year price returns)

In [47]:
symbol = 'GOOGL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data



{'companyName': 'Alphabet Inc',
 'marketcap': 1909838913840,
 'week52high': 3073.25,
 'week52low': 1748.61,
 'week52highSplitAdjustOnly': 3166.27,
 'week52lowSplitAdjustOnly': 1791.54,
 'week52change': 0.6300097459941708,
 'sharesOutstanding': 309759313,
 'float': 0,
 'avg10Volume': 1587243,
 'avg30Volume': 1466041,
 'day200MovingAvg': 2839.13,
 'day50MovingAvg': 2968.93,
 'employees': 136041,
 'ttmEPS': 105.4,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '',
 'exDividendDate': '',
 'nextEarningsDate': '2022-01-19',
 'peRatio': 26.955990930845946,
 'beta': 0,
 'maxChangePercent': 11.10796487289822,
 'year5ChangePercent': 2.47072855671931,
 'year2ChangePercent': 0.9656477256032612,
 'year1ChangePercent': 0.6235222597847441,
 'ytdChangePercent': -0.03780053772680683,
 'month6ChangePercent': 0.09858180573802258,
 'month3ChangePercent': -0.01354094149433568,
 'month1ChangePercent': -0.04847313705689925,
 'day30ChangePercent': -0.03478051231059828,
 'day5ChangePercent': 

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year2ChangePercent']

0.9511610571126501

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [50]:
my_columns = ['Ticker','Stock Price','Year 2 Change Percent', 'Number of Shares to Buy']

dataFrame = pd.DataFrame(columns = my_columns) # two dimensional 
dataFrame

# will yield n number of chunks that forms a list
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]
    
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
dataFrame = pd.DataFrame(columns = my_columns)

# error with WLTW stock
for symbol_string in symbol_strings[:4]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print(symbol_string.split(','))
    # print(data)
    for symbol in symbol_string.split(','):
        dataFrame = dataFrame.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'],
                    data[symbol]['stats']['year2ChangePercent'],
                    'N/A'
                ],
                index = dataFrame.columns
            ),
            ignore_index = True
        )

dataFrame

,Ticker,Stock Price,Year 2 Change Percent,Number of Shares to Buy
0,A,147.46,0.663499,N/A
1,AAL,19.40,-0.332149,N/A
2,AAP,244.30,0.662812,N/A
3,AAPL,181.39,1.287303,N/A
4,ABBV,142.48,0.680677,N/A
...,...,...,...,...
395,RHI,119.38,0.924193,N/A
396,RJF,111.70,0.821472,N/A
397,RL,117.19,-0.002367,N/A
398,RMD,261.68,0.628172,N/A


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [57]:
dataFrame.sort_values('Year 2 Change Percent', ascending = False, inplace = True) # inplace modifies the orginal pands dataframe
dataFrame = dataFrame[:50]
dataFrame.reset_index(inplace = True)
dataFrame

,level_0,index,Ticker,Stock Price,Year 2 Change Percent,Number of Shares to Buy
0,0,345,NVDA,280.75,3.512676,N/A
1,1,78,CARR,53.60,3.405014,N/A
2,2,275,LB,83.10,2.550844,N/A
3,3,179,FCX,44.85,2.466801,N/A
4,4,299,MCHP,85.79,2.282088,N/A
5,5,23,ALB,244.06,2.04305,N/A
6,6,174,F,25.35,1.88126,N/A
7,7,31,AMD,143.48,1.845865,N/A
8,8,29,AMAT,171.00,1.803395,N/A
9,9,385,PWR,111.14,1.694121,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: